---
title: "Post 4 - Perceptron -- CHANGE"
author: Col McDermott
date: "03-26-2025"
description: "An introductory exploration of the Perceptron algorithm -- CHANGE."
format: html
code-fold: true
execute:
  warning: false
  message: false
---

# Abstract



### Accessing the Data

In [43]:
# Includuing all additional imports
from sklearn.model_selection import cross_val_score, train_test_split
from matplotlib.colors import LinearSegmentedColormap
from folktables import ACSDataSource, BasicProblem
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from matplotlib import pyplot as plt
from sklearn.svm import SVC
import seaborn as sns
import pandas as pd
import numpy as np
import itertools

# Retrieving the data

*Including all additional imports*

Description of the Data.

### Exploring the data

In [44]:
# Data exploration

*Code description*

Description of exploring the data.

### Constructing the Model

#### Data Preparation

In [45]:
# Preparing the data for the model

*Code description*

Description of the data preparation process.

#### Fitting the Model

In [46]:
# Fitting the model

*Code description*

Description of fitting the model.

#### Refining the Model

In [47]:
# Refining the model

*Code description*

Analysis of refining the model.

### Model Accuracy Analysis

In [48]:
# Calculating various accuracy metrics

*Code description*

Analysis of the model accuracy and performance.

# Discussion

Analysis of findings from this study.

*During the implementation process of this replication study, I collaborated with _____.*